# View frozen graph with tensorboard

In [1]:
import numpy as np
import tensorflow as tf

Parse GraphDef from `frozen_graph.pb` file

In [2]:
#frozen_graph_file = "./frozen_model/frozen_graph.pb"
frozen_graph_file = "./ssd_mobilenet_v2_coco_2018_03_29/frozen_inference_graph.pb"
# reset the default graph to clean up this notebook
tf.compat.v1.reset_default_graph()
with tf.compat.v1.gfile.FastGFile(frozen_graph_file, 'rb') as f:
    graph_def = tf.compat.v1.GraphDef()
    # Parses a serialized binary message into the current message.
    graph_def.ParseFromString(f.read())

Instructions for updating:
Use tf.gfile.GFile.


Run the graph once to produce log events.
Afterwards the graph can be viewed with the command
```
tensorboard --logdir=frozen_logs
```

In [3]:
def analyze_inputs_outputs(graph):
    ops = graph.get_operations()
    outputs_set = set(ops)
    inputs = []
    for op in ops:
        if len(op.inputs) == 0 and op.type != 'Const':
            inputs.append(op)
        else:
            for input_tensor in op.inputs:
                if input_tensor.op in outputs_set:
                    outputs_set.remove(input_tensor.op)
    outputs = list(outputs_set)
    return (inputs, outputs)

with tf.compat.v1.Session() as sess:
    # Returns a context manager that makes this `Graph` the default graph.
    sess.graph.as_default()
    # Import a serialized TensorFlow `GraphDef` protocol buffer and place it into the current default `Graph`
    tf.import_graph_def(graph_def)

    (input_ops, output_ops) = analyze_inputs_outputs(sess.graph)
    # The output tensor is the output of the last operation
    output_tensor = output_ops[0].outputs[0]
    # The input tensor is the output of the first operation
    input_tensor = input_ops[0].outputs[0]

    # Input/Output Tensor names
    print("Input tensor: ", input_tensor.name)
    print("Output tensor: ", output_tensor.name)

    # Dummy input
    dummy_input = np.zeros((1, 160, 160, 3), dtype=input_tensor.dtype.as_numpy_dtype)

    # Run the graph once and write the events into the `frozen_logs` folder
    with tf.compat.v1.summary.FileWriter("frozen_logs", sess.graph) as writer:
        sess.run(output_tensor, {input_tensor.name: dummy_input})

KeyboardInterrupt: 